In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations 

In [3]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_mftip_frame_id = get_compiled_frameID(m, "rh_ffmiddle")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
display(mftip_transform.origin)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.033
 -0.01
  0.38701

### Hand Visualization

In [25]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.045, -0.01, 0.38701]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.045
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.38701
 0.0  0.0  0.0   1.0

In [26]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_ffmiddle")
display_frame(m, ls, "world")
display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [ ]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))
show(IOContext(stdout, :limit => false),  MIME("text/plain"), keys(joints(robot)))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [3]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiples springs/dampers

In [4]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

#Lightly constraint some joints to avoid unwanted motions
add_component!(vms, LinearSpring(1.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")

# PHASE 1 : FINGERS SPACING

add_coordinate!(vms, ConstCoord(0.0); id = "angular spring length")

#ff mf spacing
add_coordinate!(vms, CoordDifference(".robot.rh_MFJ4_coord", ".robot.rh_FFJ4_coord"); id="ff mf j4 angular diff")
add_coordinate!(vms, CoordDifference("ff mf j4 angular diff", "angular spring length") ; id="ff mf j4 angular error")
add_component!(vms, LinearSpring(10.0, "ff mf j4 angular error"); id="ff mf angular spring")
add_component!(vms, LinearDamper(10.0, "ff mf j4 angular error"); id="ff mf angular damper")

#mf rf spacing
add_coordinate!(vms, CoordSum(".robot.rh_RFJ4_coord", ".robot.rh_MFJ4_coord"); id="mf rf j4 angular diff")
add_coordinate!(vms, CoordSum("mf rf j4 angular diff", "angular spring length") ; id="mf rf j4 angular error")
add_component!(vms, LinearSpring(10.0, "mf rf j4 angular error"); id="mf rf angular spring")
add_component!(vms, LinearDamper(10.0, "mf rf j4 angular error"); id="mf rf angular damper")

#rf lf spacing
add_coordinate!(vms, CoordDifference(".robot.rh_RFJ4_coord", ".robot.rh_LFJ4_coord"); id="rf lf j4 angular diff")
add_coordinate!(vms, CoordDifference("rf lf j4 angular diff", "angular spring length") ; id="rf lf j4 angular error")
add_component!(vms, LinearSpring(10.0, "rf lf j4 angular error"); id="rf lf angular spring")
add_component!(vms, LinearDamper(10.0, "rf lf j4 angular error"); id="rf lf angular damper")

#th spacing
add_coordinate!(vms, ConstCoord(0.0); id="th spring length")
add_coordinate!(vms, CoordDifference(".robot.rh_THJ4_coord", "th spring length"); id="th j4 error")
add_component!(vms, LinearSpring(1.0, "th j4 error"); id="th j4 spring")

# PHASE 2 : GRASPING : EACH JOINTS SHOULD GO TO ITS FINAL POSITION, BUT AT DIFFERENT TIMESCALES

add_coordinate!(vms, ConstCoord(0.0); id="J3 target angle") # First that should be activated
add_coordinate!(vms, ConstCoord(0.0); id="J2 target angle") # Second 
add_coordinate!(vms, ConstCoord(0.0); id="J1 target angle") # Last

# First finger
add_coordinate!(vms, CoordDifference(".robot.rh_FFJ3_coord", "J3 target angle"); id="ff j3 angle error")
add_component!(vms, LinearSpring(1.0, "ff j3 angle error"); id="ff j3 spring")
add_component!(vms, LinearDamper(1.0, "ff j3 angle error"); id="ff j3 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_FFJ2_coord", "J2 target angle"); id="ff j2 angle error")
add_component!(vms, LinearSpring(1.0, "ff j2 angle error"); id="ff j2 spring")
add_component!(vms, LinearDamper(1.0, "ff j2 angle error"); id="ff j2 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_FFJ1_coord", "J1 target angle"); id="ff j1 angle error")
add_component!(vms, LinearSpring(1.0, "ff j1 angle error"); id="ff j1 spring")
add_component!(vms, LinearDamper(1.0, "ff j1 angle error"); id="ff j1 damper")

# Middle Finger
add_coordinate!(vms, CoordDifference(".robot.rh_MFJ3_coord", "J3 target angle"); id="mf j3 angle error")
add_component!(vms, LinearSpring(1.0, "mf j3 angle error"); id="mf j3 spring")
add_component!(vms, LinearDamper(1.0, "mf j3 angle error"); id="mf j3 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_MFJ2_coord", "J2 target angle"); id="mf j2 angle error")
add_component!(vms, LinearSpring(1.0, "mf j2 angle error"); id="mf j2 spring")
add_component!(vms, LinearDamper(1.0, "mf j2 angle error"); id="mf j2 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_MFJ1_coord", "J1 target angle"); id="mf j1 angle error")
add_component!(vms, LinearSpring(1.0, "mf j1 angle error"); id="mf j1 spring")
add_component!(vms, LinearDamper(1.0, "mf j1 angle error"); id="mf j1 damper")

# Ring Finger
add_coordinate!(vms, CoordDifference(".robot.rh_RFJ3_coord", "J3 target angle"); id="rf j3 angle error")
add_component!(vms, LinearSpring(1.0, "rf j3 angle error"); id="rf j3 spring")
add_component!(vms, LinearDamper(1.0, "rf j3 angle error"); id="rf j3 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_RFJ2_coord", "J2 target angle"); id="rf j2 angle error")
add_component!(vms, LinearSpring(1.0, "rf j2 angle error"); id="rf j2 spring")
add_component!(vms, LinearDamper(1.0, "rf j2 angle error"); id="rf j2 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_RFJ1_coord", "J1 target angle"); id="rf j1 angle error")
add_component!(vms, LinearSpring(1.0, "rf j1 angle error"); id="rf j1 spring")
add_component!(vms, LinearDamper(1.0, "rf j1 angle error"); id="rf j1 damper")

# Little Finger
add_coordinate!(vms, CoordDifference(".robot.rh_LFJ3_coord", "J3 target angle"); id="lf j3 angle error")
add_component!(vms, LinearSpring(1.0, "lf j3 angle error"); id="lf j3 spring")
add_component!(vms, LinearDamper(1.0, "lf j3 angle error"); id="lf j3 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_LFJ2_coord", "J2 target angle"); id="lf j2 angle error")
add_component!(vms, LinearSpring(1.0, "lf j2 angle error"); id="lf j2 spring")
add_component!(vms, LinearDamper(1.0, "lf j2 angle error"); id="lf j2 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_LFJ1_coord", "J1 target angle"); id="lf j1 angle error")
add_component!(vms, LinearSpring(1.0, "lf j1 angle error"); id="lf j1 spring")
add_component!(vms, LinearDamper(1.0, "lf j1 angle error"); id="lf j1 damper")

# Thumb 
add_coordinate!(vms, ConstCoord(0.0); id="th J5 target angle") # First that should be activated
add_coordinate!(vms, ConstCoord(0.0); id="th J2 target angle") # Second 
add_coordinate!(vms, ConstCoord(0.0); id="th J1 target angle") # Last

add_coordinate!(vms, CoordDifference(".robot.rh_THJ5_coord", "th J5 target angle"); id="th j5 angle error")
add_component!(vms, LinearSpring(1.0, "th j5 angle error"); id="th j5 spring")
add_component!(vms, LinearDamper(1.0, "th j5 angle error"); id="th j5 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_THJ2_coord", "th J2 target angle"); id="th j2 angle error")
add_component!(vms, LinearSpring(1.0, "th j2 angle error"); id="th j2 spring")
add_component!(vms, LinearDamper(1.0, "th j2 angle error"); id="th j2 damper")
add_coordinate!(vms, CoordDifference(".robot.rh_THJ1_coord", "th J1 target angle"); id="th j1 angle error")
add_component!(vms, LinearSpring(1.0, "th j1 angle error"); id="th j1 spring")
add_component!(vms, LinearDamper(1.0, "th j1 angle error"); id="th j1 damper")


"th j1 damper"

## Simulating the Robot

### Setting Up the Simulation

In [11]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the lateral pinch motion while 1 being 
the end
"""
function update_power_sphere_coord(args, cache, coord)
    angular_spring_length_id, th_spring_length_id, j3_target_angle_id, j2_target_angle_id, j1_target_angle_id, th_j5_target_angle_id,
    th_j2_target_angle_id, th_j1_target_angle_id = args

    # PHASE 1 : FINGERS SPACING
    #update the angle spacing the fingers, and the thumb
    spacing_end_coord = 0.3
    if coord < spacing_end_coord
        fingers_angle_min = 0.0
        fingers_angle_max = 0.5
        fingers_angle_value = fingers_angle_min + (fingers_angle_max - fingers_angle_min)*(coord/spacing_end_coord)
        cache[angular_spring_length_id] = remake(cache[angular_spring_length_id] ; coord_data = ConstCoord(fingers_angle_value))
        th_angle_min = 0.0
        th_angle_max = 1.22
        th_angle_value = th_angle_min + (th_angle_max - th_angle_min)*(coord/spacing_end_coord)
        cache[th_spring_length_id] = remake(cache[th_spring_length_id] ; coord_data = ConstCoord(th_angle_value))
    end

    # PHASE 2 : GRASPING

    #Start by metacarpophalangeal joints (MCP joints)
    j3_start_coord = 0.3
    j3_end_coord = 0.8
    if coord >= j3_start_coord && coord <= j3_end_coord

        j3_coord_value = (coord - j3_start_coord)/(j3_end_coord - j3_start_coord) 

        #fingers 
        j3_angle_min = 0.0
        j3_angle_max = 1.57
        j3_angle_value = j3_angle_min + (j3_angle_max - j3_angle_min)*j3_coord_value
        cache[j3_target_angle_id] = remake(cache[j3_target_angle_id]; coord_data = ConstCoord(j3_angle_value))

    end

    #shift metacarpophalangeal joints (MCP joints) of the thumb
    th_j3_start_coord = 0.6
    th_j3_end_coord = 0.9
    if coord >= th_j3_start_coord && coord <= th_j3_end_coord

        th_j3_coord_value = (coord - th_j3_start_coord)/(j3_end_coord - th_j3_start_coord) 

        #thumb
        th_j5_angle_min = 0.0
        th_j5_angle_max = 1.05
        th_j5_angle_value = th_j5_angle_min + (th_j5_angle_max - th_j5_angle_min)*th_j3_coord_value
        cache[th_j5_target_angle_id] = remake(cache[th_j5_target_angle_id] ; coord_data = ConstCoord(th_j5_angle_value))
    end

    #Followed by proximal interphalangeal joints (PIP joints)
    j2_start_coord = 0.5
    j2_end_coord = 0.9
    if coord >= j2_start_coord && coord <= j2_end_coord

        j2_coord_value = (coord - j2_start_coord)/(j2_end_coord - j2_start_coord) 

        #fingers
        j2_angle_min = 0.0
        j2_angle_max = 1.57
        j2_angle_value = j2_angle_min + (j2_angle_max - j2_angle_min)*j2_coord_value
        cache[j2_target_angle_id] = remake(cache[j2_target_angle_id]; coord_data = ConstCoord(j2_angle_value))

        #thumb
        th_j2_angle_min = 0.0
        th_j2_angle_max = 0.7
        th_j2_angle_value = th_j2_angle_min + (th_j2_angle_max - th_j2_angle_min)*j2_coord_value
        cache[th_j2_target_angle_id] = remake(cache[th_j2_target_angle_id] ; coord_data = ConstCoord(th_j2_angle_value))
    end

    #And then distal interphalangeal joints (DIP joints)
    j1_start_coord = 0.7
    j1_end_coord = 1.0
    if coord >= j1_start_coord && coord <= j1_end_coord

        j1_coord_value = (coord - j1_start_coord)/(j1_end_coord - j1_start_coord) 

        #fingers
        j1_angle_min = 0.0
        j1_angle_max = 1.57
        j1_angle_value = j1_angle_min + (j1_angle_max - j1_angle_min)*j1_coord_value
        cache[j1_target_angle_id] = remake(cache[j1_target_angle_id]; coord_data = ConstCoord(j1_angle_value))

        #thumb
        th_j1_angle_min = 0.0
        th_j1_angle_max = 1.57
        th_j1_angle_value = th_j1_angle_min + (th_j1_angle_max - th_j1_angle_min)*j1_coord_value
        cache[th_j1_target_angle_id] = remake(cache[th_j1_target_angle_id] ; coord_data = ConstCoord(th_j1_angle_value))
    end

    nothing
end

function f_setup(cache) 

    return  (get_compiled_coordID(cache, "angular spring length"), get_compiled_coordID(cache, "th spring length"), get_compiled_coordID(cache, "J3 target angle"),
            get_compiled_coordID(cache, "J2 target angle"), get_compiled_coordID(cache, "J1 target angle"), get_compiled_coordID(cache, "th J5 target angle"),
            get_compiled_coordID(cache, "th J2 target angle"), get_compiled_coordID(cache, "th J1 target angle"))
end

function f_control(cache, t, args, extra)

    t_start = 0.
    t_end = 4. #we want to reach the end position at t = t_end 
    
    if t >= t_start && t <= t_end 
        coord_value = (t - t_start) /(t_end -t_start)
        update_power_sphere_coord(args, cache, coord_value)
    end
    nothing
end

tspan = (0., 5.)
vms_compiled = compile(vms)

q_init = generate_q_init(vms_compiled)
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with power sphere"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-3, reltol=1e-3);

# Rosenbrock23 : 2.58
# Rodas5P : 3.21
# TRBDF2 : > 1h
# QNDF : > 1h
# FBDF : > 1h
# Kvaerno5 : > 1h
# KenCarp4 : > 1h

[ Info: Simulating shadow robot with power sphere
ODE   0%|█                                              |  ETA: N/A
ODE  21%|██████████                                     |  ETA: 0:00:33
ODE  43%|█████████████████████                          |  ETA: 0:00:30
ODE  58%|████████████████████████████                   |  ETA: 0:00:29
ODE  67%|████████████████████████████████               |  ETA: 0:00:30
ODE  76%|████████████████████████████████████           |  ETA: 0:00:24
ODE 100%|███████████████████████████████████████████████| Time: 0:01:24


### Visualizing the Results

In [12]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_power_sphere.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);